# Conexão com o JUSBR 

Este notebook demonstra como autenticar e obter um token de acesso para o portal JUSBR usando a classe `JUSBR_Scraper`.


In [ ]:
import juscraper as jus
import json

## Autenticação manual via gov.br

1. Acesse https://www.jus.br
2. Faça login usando o gov.br
3. Entre na página https://portaldeservicos.pdpj.jus.br/consulta (ou clique no botão 'Consultar processo', que aparece após o login)
4. Na nova página, abra a aba Network do navegador (F12 ou 'Inspecionar elemento')
5. Atualize a página (F5 ou ctrl+R ou no botão atualizar)
6. Nas requisições que vão aparecer, procure a requisição que tem nome 'token'. Clique nela.
7. Na tela ao lado, clique em 'Resposta'.
8. Selecione e copie o campo 'access_token' que aparece lá.


In [ ]:
jusbr = jus.scraper("jusbr")

token = input("Digite o token JWT: ")

jusbr.auth(token)

## Baixando metadados dos processos

Baixando uma lista de casos:

In [ ]:
processos = ['10092161720238260016', '00039417120248260048']

resultados = jusbr.cpopg(processos)

In [ ]:
resultados

In [ ]:
result_list = resultados['detalhes'].iloc[0]

print(json.dumps(result_list, indent=2, ensure_ascii=False))

Se quiser visualizar o texto do caso:

## Baixando autos dos processos

In [ ]:
df_docs = jusbr.download_documents(resultados)

df_docs

In [ ]:
print(df_docs['texto'][0])

## Baixando uma lista grande de processos

Se a lista de processos é muito grande, pode ser uma boa prática baixar um processo por vez e salvar os resultados em um arquivo parquet, por exemplo.

No script abaixo, criamos uma pasta dentro da pasta atual, chamada jusbr/cpopg/. Ali dentro, para cada processo, criamos uma pasta com o número do processo e salvamos 2 arquivos: 1 com os metadados daquele processo e 1 com os textos dos documentos. A chave para juntar as duas bases é o número do processo.

In [ ]:
import os
import pandas as pd
from tqdm import tqdm

processos = ['10092161720238260016', '00039417120248260048', '10226951920238260003']

resultados = jusbr.cpopg(processos)

for i, row in tqdm(resultados.iterrows()):
    processo = row['processo']
    row_df = pd.DataFrame([row])
    # cria a pasta
    process_path = os.path.join('jusbr/cpopg', processo)
    os.makedirs(process_path, exist_ok=True)
    # baixa documentos do processo
    df_docs = jusbr.download_documents(row_df)
    # salva documentos
    df_docs.to_parquet(os.path.join(process_path, 'documents.parquet'), index=False)
    # salva metadata
    row_df.to_parquet(os.path.join(process_path, 'metadata.parquet'), index=False)